Xception

In [1]:
import glob
import os
import cv2
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from google.colab.patches import cv2_imshow
from sklearn.model_selection import train_test_split
from matplotlib import cm
from PIL import Image, ImageOps
from tensorflow.python.keras.callbacks import EarlyStopping
from tensorflow.python.keras.callbacks import ModelCheckpoint
from keras.applications.xception import Xception
from keras import backend as K
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import mean_squared_error
from tensorflow.python.keras.engine.sequential import Sequential
from tensorflow.python.keras.layers import Dense, Dropout, Activation, Flatten, InputLayer, MaxPooling2D, AveragePooling2D
from tensorflow.python.keras.layers import Convolution2D
from tensorflow.python.keras import layers, models, Model
from tensorflow.keras.preprocessing import image_dataset_from_directory

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!unzip ./drive/MyDrive/data/mrdata.zip

Archive:  ./drive/MyDrive/data/mrdata.zip
   creating: mrdata/
   creating: mrdata/10/
   creating: mrdata/10/depth/
  inflating: mrdata/10/depth/11.29.24.441.png  
  inflating: mrdata/10/depth/11.29.24.554.png  
  inflating: mrdata/10/depth/11.29.24.663.png  
  inflating: mrdata/10/depth/11.29.24.772.png  
  inflating: mrdata/10/depth/11.29.24.883.png  
  inflating: mrdata/10/depth/11.29.24.997.png  
  inflating: mrdata/10/depth/11.29.25.104.png  
  inflating: mrdata/10/depth/11.29.25.210.png  
  inflating: mrdata/10/depth/11.29.25.322.png  
  inflating: mrdata/10/depth/11.29.25.433.png  
  inflating: mrdata/10/depth/11.29.25.543.png  
  inflating: mrdata/10/depth/11.29.25.651.png  
  inflating: mrdata/10/depth/11.29.25.764.png  
  inflating: mrdata/10/depth/mirror11.29.24.405.png  
  inflating: mrdata/10/depth/mirror11.29.24.489.png  
  inflating: mrdata/10/depth/mirror11.29.24.565.png  
  inflating: mrdata/10/depth/mirror11.29.24.646.png  
  inflating: mrdata/10/depth/mirror11.29.24

In [4]:
W = 512
H = 424
max_value_for_normalization = -1

def make_df(path_to_xlsx, path_to_folder):
    if not os.path.exists(path_to_xlsx):
        print('[Err] Fail xlsx path')
        return
    if not os.path.exists(path_to_folder):
        print('[Err] Fail folder path')
        return

    df = pd.read_excel(path_to_xlsx)
    df = df.fillna(0)
    df['Body length (cm)'] = df['Body length (cm)'].astype(int)
    df['Chest girth (cm)'] = df['Chest girth (cm)'].astype(int)
    df['Withers height (cm)'] = df['Withers height (cm)'].astype(int)
    df['Chest depth (cm)'] = df['Chest depth (cm)'].astype(int)
    df['Chest width (cm)'] = df['Chest width (cm)'].astype(int)
    df['Live weight (kg)'] = df['Live weight (kg)'].astype(int)
    df.drop(
        columns=['Frame', 'Content', 'Number of pig', 'Left', 'Right', 'Quality', 'Transform\nT_right_to_left', 'Sex', 'Age (days)'], inplace=True)

    df = df.iloc[2:]
    df['image'] = None
    for index, row in df.iterrows():
        path = f'{path_to_folder}/{index}/depth'
        if not os.path.exists(path):
            continue
        files = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]
        print(f'For index {index} count {len(files)} files')
        for file in files:
            raw_image = cv2.imread(f'{path}/{file}', cv2.CV_16U).astype(np.float32)
            raw_image = cv2.resize(raw_image, (W, H))
            raw_image = np.repeat(raw_image[..., np.newaxis], 3, -1)
            raw_image = raw_image.reshape(-1, W, H, 3)

            global max_value_for_normalization
            if np.amax(raw_image) > max_value_for_normalization:
                max_value_for_normalization = np.amax(raw_image)

            new_data = {
                'image': raw_image,
                'Live weight (kg)': df.iloc[index - 3]['Live weight (kg)'],
                'Body length (cm)': df.iloc[index - 3]['Body length (cm)'],
                'Chest girth (cm)': df.iloc[index - 3]['Chest girth (cm)'],
                'Withers height (cm)': df.iloc[index - 3]['Withers height (cm)'],
                'Chest depth (cm)': df.iloc[index - 3]['Chest depth (cm)'],
                'Chest width (cm)': df.iloc[index - 3]['Chest width (cm)'],
            }

            df = pd.concat([df, pd.DataFrame.from_records([new_data])])

    df['image'] = df['image'] / max_value_for_normalization
    df = df.iloc[16:]
    df = df.reset_index().drop(columns=['index'])
    return df
  
dataframe = make_df('./drive/MyDrive/data/data.xlsx', 'mrdata')
dataframe


For index 3 count 31 files
For index 4 count 32 files
For index 5 count 21 files
For index 6 count 31 files
For index 7 count 26 files
For index 8 count 39 files
For index 9 count 27 files
For index 10 count 27 files
For index 11 count 24 files
For index 12 count 27 files
For index 13 count 17 files
For index 14 count 37 files
For index 15 count 22 files
For index 16 count 33 files
For index 17 count 22 files


,Live weight (kg),Body length (cm),Chest girth (cm),Withers height (cm),Chest depth (cm),Chest width (cm),image
0,117,114,117,55,37,32,"[[[[0. 0. 0.], [0. 0. 0.], [0. 0. 0.], [0. 0. ..."
1,117,114,117,55,37,32,"[[[[0. 0. 0.], [0. 0. 0.], [0. 0. 0.], [0. 0. ..."
2,117,114,117,55,37,32,"[[[[0. 0. 0.], [0. 0. 0.], [0. 0. 0.], [0. 0. ..."
3,117,114,117,55,37,32,"[[[[0. 0. 0.], [0. 0. 0.], [0. 0. 0.], [0. 0. ..."
4,117,114,117,55,37,32,"[[[[0. 0. 0.], [0. 0. 0.], [0.90225 0.90225 0...."
...,...,...,...,...,...,...,...
411,118,123,117,63,37,32,"[[[[0. 0. 0.], [0. 0. 0.], [0. 0. 0.], [0. 0. ..."
412,118,123,117,63,37,32,"[[[[0. 0. 0.], [0. 0. 0.], [0. 0. 0.], [0. 0. ..."
413,118,123,117,63,37,32,"[[[[0. 0. 0.], [0. 0. 0.], [0. 0. 0.], [0. 0. ..."
414,118,123,117,63,37,32,"[[[[0. 0. 0.], [0. 0. 0.], [0. 0. 0.], [0.1126..."


In [5]:
columns_features_all = ['Live weight (kg)', 'Body length (cm)', 'Chest girth (cm)', 'Withers height (cm)', 'Chest depth (cm)', 'Chest width (cm)']
columns_features_kg = ['Live weight (kg)']
columns_features_sm = ['Body length (cm)', 'Chest girth (cm)', 'Withers height (cm)', 'Chest depth (cm)', 'Chest width (cm)']
# column_img = ['image']

In [6]:
xc_nn = Xception(input_shape=(W, H, 3), include_top=False, weights='imagenet')
xc_nn.trainable = False
# xc_nn.summary()

83697664/83683744 [==============================] - 0s 0us/step


ADAM ALL PARAMS

In [7]:
!rm -r './model/best'

count = 1
evals_models = []

ss = ShuffleSplit(n_splits=4, test_size=0.2, random_state=0)
for train_ind, valid_ind in ss.split(dataframe):

  path = './model/best'
  es = EarlyStopping(monitor='val_loss', verbose=1, min_delta=0.02, patience=6)
  mc = ModelCheckpoint(path, monitor='val_loss', mode='min', verbose=1, save_best_only=True)

  print("Split dataframe")
  y_train = np.asarray([np.asarray(i) for i in dataframe.loc[train_ind][columns_features_all].values])
  X_train = np.asarray([np.asarray(i) for i in dataframe.loc[train_ind]['image'].values]).reshape(-1, W, H, 3)
  
  y_valid = np.asarray([np.asarray(i) for i in dataframe.loc[valid_ind][columns_features_all].values])
  X_valid = np.asarray([np.asarray(i) for i in dataframe.loc[valid_ind]['image'].values]).reshape(-1, W, H, 3)

  print("Create model")
  model = Sequential()
  model.add(xc_nn)
  model.add(Flatten())
  model.add(Dense(64, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(6, activation='relu'))
  model.build(input_shape=(None, W, H, 3))
  model.compile(optimizer='adam', loss='mse', metrics=['mae', 'mape'])

  print("Fit")
  history = model.fit(X_train, y_train, epochs=40, validation_split=0.2, validation_data=(X_valid, y_valid), callbacks=[es, mc], batch_size=4, verbose=0)
  
  !cp -av ./model/best '/content/drive/MyDrive/data/models/xcp_nn/best_adam_all_params_{count}'
  count = count + 1

  print("Evaluate")
  model = keras.models.load_model(path)
  score = model.evaluate(X_train, y_train, verbose=1)
  evals_models.append(score)
  score = model.evaluate(X_valid, y_valid, verbose=1)
  evals_models.append(score)

  !rm -r './model/best'

rm: cannot remove './model/best': No such file or directory
Split dataframe
Create model
Fit

Epoch 00001: val_loss improved from inf to 1057.63977, saving model to ./model/best
INFO:tensorflow:Assets written to: ./model/best/assets


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-7-4e0b79f37345>", line 31, in <module>
    history = model.fit(X_train, y_train, epochs=40, validation_split=0.2, validation_data=(X_valid, y_valid), callbacks=[es, mc], batch_size=4, verbose=0)
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py", line 1189, in fit
    tmp_logs = self.train_function(iterator)
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/traceback_utils.py", line 150, in error_handler
    return fn(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/eager/def_function.py", line 915, in __call__
    result = self._call(*args, **kwds)
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/eager/def_function.py", line 947, in _call
    return self._stateless

KeyboardInterrupt: ignored

In [ ]:
evals_models

RMSPROP ALL PARAMS

In [ ]:
!rm -r './model/best'

count = 1
evals_models = []

ss = ShuffleSplit(n_splits=4, test_size=0.2, random_state=0)
for train_ind, valid_ind in ss.split(dataframe):

  path = './model/best'
  es = EarlyStopping(monitor='val_loss', verbose=1, min_delta=0.02, patience=6)
  mc = ModelCheckpoint(path, monitor='val_loss', mode='min', verbose=1, save_best_only=True)

  print("Split dataframe")
  y_train = np.asarray([np.asarray(i) for i in dataframe.loc[train_ind][columns_features_all].values])
  X_train = np.asarray([np.asarray(i) for i in dataframe.loc[train_ind]['image'].values]).reshape(-1, W, H, 3)
  
  y_valid = np.asarray([np.asarray(i) for i in dataframe.loc[valid_ind][columns_features_all].values])
  X_valid = np.asarray([np.asarray(i) for i in dataframe.loc[valid_ind]['image'].values]).reshape(-1, W, H, 3)

  print("Create model")
  model = Sequential()
  model.add(xc_nn)
  model.add(Flatten())
  model.add(Dense(64, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(6, activation='relu'))
  model.build(input_shape=(None, W, H, 3))
  model.compile(optimizer='rmsprop', loss='mse', metrics=['mae', 'mape'])

  print("Fit")
  history = model.fit(X_train, y_train, epochs=40, validation_split=0.2, validation_data=(X_valid, y_valid), callbacks=[es, mc], batch_size=4, verbose=0)
  
  !cp -av ./model/best '/content/drive/MyDrive/data/models/xcp_nn/best_rmsprop_all_params_{count}'
  count = count + 1

  print("Evaluate")
  model = keras.models.load_model(path)
  score = model.evaluate(X_train, y_train, verbose=1)
  evals_models.append(score)
  score = model.evaluate(X_valid, y_valid, verbose=1)
  evals_models.append(score)

  !rm -r './model/best'

In [ ]:
evals_models

ADAM KG PARAM

In [ ]:
!rm -r './model/best'

count = 1
evals_models = []

ss = ShuffleSplit(n_splits=4, test_size=0.2, random_state=0)
for train_ind, valid_ind in ss.split(dataframe):

  path = './model/best'
  es = EarlyStopping(monitor='val_loss', verbose=1, min_delta=0.02, patience=6)
  mc = ModelCheckpoint(path, monitor='val_loss', mode='min', verbose=1, save_best_only=True)

  print("Split dataframe")
  y_train = np.asarray([np.asarray(i) for i in dataframe.loc[train_ind][columns_features_kg].values])
  X_train = np.asarray([np.asarray(i) for i in dataframe.loc[train_ind]['image'].values]).reshape(-1, W, H, 3)
  
  y_valid = np.asarray([np.asarray(i) for i in dataframe.loc[valid_ind][columns_features_kg].values])
  X_valid = np.asarray([np.asarray(i) for i in dataframe.loc[valid_ind]['image'].values]).reshape(-1, W, H, 3)

  print("Create model")
  model = Sequential()
  model.add(xc_nn)
  model.add(Flatten())
  model.add(Dense(64, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(1, activation='relu'))
  model.build(input_shape=(None, W, H, 3))
  model.compile(optimizer='adam', loss='mse', metrics=['mae', 'mape'])

  print("Fit")
  history = model.fit(X_train, y_train, epochs=40, validation_split=0.2, validation_data=(X_valid, y_valid), callbacks=[es, mc], batch_size=4, verbose=0)
  
  !cp -av ./model/best '/content/drive/MyDrive/data/models/xcp_nn/best_adam_kg_param_{count}'
  count = count + 1

  print("Evaluate")
  model = keras.models.load_model(path)
  score = model.evaluate(X_train, y_train, verbose=1)
  evals_models.append(score)
  score = model.evaluate(X_valid, y_valid, verbose=1)
  evals_models.append(score)

  !rm -r './model/best'

In [ ]:
evals_models

RMSPROP KG PARAM

In [ ]:
!rm -r './model/best'

count = 1
evals_models = []

ss = ShuffleSplit(n_splits=4, test_size=0.2, random_state=0)
for train_ind, valid_ind in ss.split(dataframe):

  path = './model/best'
  es = EarlyStopping(monitor='val_loss', verbose=1, min_delta=0.02, patience=6)
  mc = ModelCheckpoint(path, monitor='val_loss', mode='min', verbose=1, save_best_only=True)

  print("Split dataframe")
  y_train = np.asarray([np.asarray(i) for i in dataframe.loc[train_ind][columns_features_kg].values])
  X_train = np.asarray([np.asarray(i) for i in dataframe.loc[train_ind]['image'].values]).reshape(-1, W, H, 3)
  
  y_valid = np.asarray([np.asarray(i) for i in dataframe.loc[valid_ind][columns_features_kg].values])
  X_valid = np.asarray([np.asarray(i) for i in dataframe.loc[valid_ind]['image'].values]).reshape(-1, W, H, 3)

  print("Create model")
  model = Sequential()
  model.add(xc_nn)
  model.add(Flatten())
  model.add(Dense(64, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(1, activation='relu'))
  model.build(input_shape=(None, W, H, 3))
  model.compile(optimizer='rmsprop', loss='mse', metrics=['mae', 'mape'])

  print("Fit")
  history = model.fit(X_train, y_train, epochs=40, validation_split=0.2, validation_data=(X_valid, y_valid), callbacks=[es, mc], batch_size=4, verbose=0)
  
  !cp -av ./model/best '/content/drive/MyDrive/data/models/xcp_nn/best_rmsprop_kg_param_{count}'
  count = count + 1

  print("Evaluate")
  model = keras.models.load_model(path)
  score = model.evaluate(X_train, y_train, verbose=1)
  evals_models.append(score)
  score = model.evaluate(X_valid, y_valid, verbose=1)
  evals_models.append(score)

  !rm -r './model/best'

In [ ]:
evals_models

ADAM SM PARAMS

In [ ]:
!rm -r './model/best'

count = 1
evals_models = []

ss = ShuffleSplit(n_splits=4, test_size=0.2, random_state=0)
for train_ind, valid_ind in ss.split(dataframe):

  path = './model/best'
  es = EarlyStopping(monitor='val_loss', verbose=1, min_delta=0.02, patience=6)
  mc = ModelCheckpoint(path, monitor='val_loss', mode='min', verbose=1, save_best_only=True)

  print("Split dataframe")
  y_train = np.asarray([np.asarray(i) for i in dataframe.loc[train_ind][columns_features_sm].values])
  X_train = np.asarray([np.asarray(i) for i in dataframe.loc[train_ind]['image'].values]).reshape(-1, W, H, 3)
  
  y_valid = np.asarray([np.asarray(i) for i in dataframe.loc[valid_ind][columns_features_sm].values])
  X_valid = np.asarray([np.asarray(i) for i in dataframe.loc[valid_ind]['image'].values]).reshape(-1, W, H, 3)

  print("Create model")
  model = Sequential()
  model.add(xc_nn)
  model.add(Flatten())
  model.add(Dense(64, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(5, activation='relu'))
  model.build(input_shape=(None, W, H, 3))
  model.compile(optimizer='adam', loss='mse', metrics=['mae', 'mape'])

  print("Fit")
  history = model.fit(X_train, y_train, epochs=40, validation_split=0.2, validation_data=(X_valid, y_valid), callbacks=[es, mc], batch_size=4, verbose=0)
  
  !cp -av ./model/best '/content/drive/MyDrive/data/models/xcp_nn/best_adam_sm_params_{count}'
  count = count + 1

  print("Evaluate")
  model = keras.models.load_model(path)
  score = model.evaluate(X_train, y_train, verbose=1)
  evals_models.append(score)
  score = model.evaluate(X_valid, y_valid, verbose=1)
  evals_models.append(score)

  !rm -r './model/best'

In [ ]:
evals_models

RMSPROP SM PARAMS

In [ ]:
!rm -r './model/best'

count = 1
evals_models = []

ss = ShuffleSplit(n_splits=4, test_size=0.2, random_state=0)
for train_ind, valid_ind in ss.split(dataframe):

  path = './model/best'
  es = EarlyStopping(monitor='val_loss', verbose=1, min_delta=0.02, patience=6)
  mc = ModelCheckpoint(path, monitor='val_loss', mode='min', verbose=1, save_best_only=True)

  print("Split dataframe")
  y_train = np.asarray([np.asarray(i) for i in dataframe.loc[train_ind][columns_features_sm].values])
  X_train = np.asarray([np.asarray(i) for i in dataframe.loc[train_ind]['image'].values]).reshape(-1, W, H, 3)
  
  y_valid = np.asarray([np.asarray(i) for i in dataframe.loc[valid_ind][columns_features_sm].values])
  X_valid = np.asarray([np.asarray(i) for i in dataframe.loc[valid_ind]['image'].values]).reshape(-1, W, H, 3)

  print("Create model")
  model = Sequential()
  model.add(xc_nn)
  model.add(Flatten())
  model.add(Dense(64, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(5, activation='relu'))
  model.build(input_shape=(None, W, H, 3))
  model.compile(optimizer='rmsprop', loss='mse', metrics=['mae', 'mape'])

  print("Fit")
  history = model.fit(X_train, y_train, epochs=40, validation_split=0.2, validation_data=(X_valid, y_valid), callbacks=[es, mc], batch_size=4, verbose=0)
  
  !cp -av ./model/best '/content/drive/MyDrive/data/models/xcp_nn/best_rmsprop_sm_params_{count}'
  count = count + 1

  print("Evaluate")
  model = keras.models.load_model(path)
  score = model.evaluate(X_train, y_train, verbose=1)
  evals_models.append(score)
  score = model.evaluate(X_valid, y_valid, verbose=1)
  evals_models.append(score)

  !rm -r './model/best'

In [ ]:
evals_models